In [1]:
import os

os.environ['PATH'] += ':/Users/abderrazzak/google-cloud-sdk/bin'

os.environ['REGION'] = "us-central1"
os.environ['PROJECT_ID'] = "orangebot-7551"
os.environ['BUCKET_URI'] = 'gs://orange_maroc'

REGION = os.environ['REGION']
PROJECT_ID = os.environ['PROJECT_ID']
BUCKET_URI = os.environ['BUCKET_URI']

# The number of dimensions for the textembedding-gecko@003 is 768
# If other embedder is used, the dimensions would probably need to change.
DIMENSIONS = 768

# Index Constants
DISPLAY_NAME = "orange_maroc"
DEPLOYED_INDEX_ID = "orange_maroc_endpoint"

In [4]:
password = ""  # Replace with your actual password
command = f"echo {password} | sudo -S gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}"
!{command}

Password:Creating gs://orange_maroc/...


In [5]:
from google.cloud import aiplatform
from langchain_google_vertexai import VertexAIEmbeddings

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [6]:
embedding_model = VertexAIEmbeddings(model_name="textembedding-gecko@003")

I0000 00:00:1721033030.501282    8819 work_stealing_thread_pool.cc:321] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1721033030.964322    8819 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1721033032.163356   27142 tcp_posix.cc:809] IOMGR endpoint shutdown


In [5]:
# NOTE : This operation can take upto 30 seconds
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    index_update_method="STREAM_UPDATE",  # allowed values BATCH_UPDATE , STREAM_UPDATE
)

I0000 00:00:1720787370.318694  707106 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/662868193247/locations/us-central1/indexes/1270309764034723840/operations/2310908218764689408
MatchingEngineIndex created. Resource name: projects/662868193247/locations/us-central1/indexes/1270309764034723840
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/662868193247/locations/us-central1/indexes/1270309764034723840')


I0000 00:00:1720787391.894778  707106 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1720787392.688504  709600 tcp_posix.cc:809] IOMGR endpoint shutdown


I0000 00:00:1720787635.530659  709592 tcp_posix.cc:809] IOMGR endpoint shutdown


In [6]:
# Create an endpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"{DISPLAY_NAME}-endpoint", public_endpoint_enabled=True
)

I0000 00:00:1720787691.902041  707106 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/662868193247/locations/us-central1/indexEndpoints/7001984714793811968/operations/8686319739795931136
MatchingEngineIndexEndpoint created. Resource name: projects/662868193247/locations/us-central1/indexEndpoints/7001984714793811968
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/662868193247/locations/us-central1/indexEndpoints/7001984714793811968')


I0000 00:00:1720787694.020746  707106 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1720787694.725809  709594 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1720787694.731568  707106 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1720787694.733464  709597 tcp_posix.cc:809] IOMGR endpoint shutdown


In [7]:
# NOTE : This operation can take upto 20 minutes
my_index_endpoint = my_index_endpoint.deploy_index(
    index=my_index, deployed_index_id=DEPLOYED_INDEX_ID
)

my_index_endpoint.deployed_indexes

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/662868193247/locations/us-central1/indexEndpoints/7001984714793811968


I0000 00:00:1720787837.578296  707106 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/662868193247/locations/us-central1/indexEndpoints/7001984714793811968/operations/5313123618895429632
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/662868193247/locations/us-central1/indexEndpoints/7001984714793811968


I0000 00:00:1720789429.038822  707106 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1720789429.878749  709600 tcp_posix.cc:809] IOMGR endpoint shutdown


[id: "orange_maroc_endpoint"
index: "projects/662868193247/locations/us-central1/indexes/1270309764034723840"
create_time {
  seconds: 1720787838
  nanos: 358217000
}
index_sync_time {
  seconds: 1720789424
  nanos: 649213000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]

I0000 00:00:1720789672.387593  709980 tcp_posix.cc:809] IOMGR endpoint shutdown


In [8]:
my_index_endpoint.deployed_indexes

[id: "orange_maroc_endpoint"
index: "projects/662868193247/locations/us-central1/indexes/1270309764034723840"
create_time {
  seconds: 1720787838
  nanos: 358217000
}
index_sync_time {
  seconds: 1720789424
  nanos: 649213000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]

In [9]:
from langchain_google_vertexai import (
    VectorSearchVectorStore,
    VectorSearchVectorStoreDatastore,
)

In [142]:
# Input text with metadata
record_data = [
    {
        "description": "A versatile pair of dark-wash denim jeans."
        "Made from durable cotton with a classic straight-leg cut, these jeans"
        " transition easily from casual days to dressier occasions.",
        "price": 65.00,
        "color": "blue",
        "season": ["fall", "winter", "spring"],
    },
    {
        "description": "A lightweight linen button-down shirt in a crisp white."
        " Perfect for keeping cool with breathable fabric and a relaxed fit.",
        "price": 34.99,
        "color": "white",
        "season": ["summer", "spring"],
    },
    {
        "description": "A soft, chunky knit sweater in a vibrant forest green. "
        "The oversized fit and cozy wool blend make this ideal for staying warm "
        "when the temperature drops.",
        "price": 89.99,
        "color": "green",
        "season": ["fall", "winter"],
    },
    {
        "description": "A classic crewneck t-shirt in a soft, heathered blue. "
        "Made from comfortable cotton jersey, this t-shirt is a wardrobe essential "
        "that works for every season.",
        "price": 19.99,
        "color": "blue",
        "season": ["fall", "winter", "summer", "spring"],
    },
    {
        "description": "A flowing midi-skirt in a delicate floral print. "
        "Lightweight and airy, this skirt adds a touch of feminine style "
        "to warmer days.",
        "price": 45.00,
        "color": "white",
        "season": ["spring", "summer"],
    },
]



# Parse and prepare input data

texts = []
metadatas = []
for record in record_data:
    record = record.copy()
    page_content = record.pop("description")
    texts.append(page_content)
    if isinstance(page_content, str):
        metadata = {**record}
        metadatas.append(metadata)

datastore_client_kwargs = {
    'database': '(default)',
    # add other necessary parameters here
}

In [143]:
# Parse and prepare input data

texts = []
metadatas = []
for record in record_data:
    record = record.copy()
    page_content = record.pop("description")
    texts.append(page_content)
    if isinstance(page_content, str):
        metadata = {**record}
        metadatas.append(metadata)

In [13]:
datastore_client_kwargs = {
    'database': '(default)',
    # add other necessary parameters here
}

In [170]:
vector_store = VectorSearchVectorStoreDatastore.from_components(
    project_id=PROJECT_ID,
    region=REGION,
    index_id="502362464684343296",
    endpoint_id="6559915069730193408",
    embedding=embedding_model,
    datastore_client_kwargs=datastore_client_kwargs,
    stream_update=True,
)

I0000 00:00:1720981422.517447   31107 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1720981422.826684  269258 subchannel.cc:806] subchannel 0x12b8fe020 {address=ipv6:%5B2a00:1450:4006:806::200a%5D:443, args={grpc.client_channel_factory=0x11dab2880, grpc.default_authority=europe-west4-aiplatform.googleapis.com:443, grpc.dns_enable_srv_queries=1, grpc.http2_scheme=https, grpc.internal.channel_credentials=0x12bb32830, grpc.internal.client_channel_call_destination=0x1187cb598, grpc.internal.event_engine=0x12b8b8c20, grpc.internal.security_connector=0x12b88c5d0, grpc.internal.subchannel_pool=0x108b8aaf0, grpc.max_receive_message_length=-1, grpc.max_send_message_length=-1, grpc.primary_user_agent=grpc-python/1.65.0, grpc.resource_quota=0x11dab1ed0, grpc.server_uri=dns:///europe-west4-aiplatform.googleapis.com:443}}: connect failed (UNKNOWN:connect: No route to host (65) {created_time:"2024-07-14T19:23:42.826495+01:00"}), backing

I0000 00:00:1720981479.717983  271022 tcp_posix.cc:809] IOMGR endpoint shutdown


In [171]:
vector_store.add_texts(texts=texts, metadatas=metadatas, is_complete_overwrite=True)

I0000 00:00:1720982041.820806  270947 subchannel.cc:806] subchannel 0x12b8fcc70 {address=ipv6:%5B2a00:1450:4006:803::200a%5D:443, args={grpc.client_channel_factory=0x11dab2880, grpc.default_authority=datastore.googleapis.com:443, grpc.http2_scheme=https, grpc.internal.channel_credentials=0x119813050, grpc.internal.client_channel_call_destination=0x1187cb598, grpc.internal.event_engine=0x12b892390, grpc.internal.security_connector=0x12b8c1080, grpc.internal.subchannel_pool=0x108b8aaf0, grpc.primary_user_agent=gcloud-python/2.4.1 grpc-python/1.65.0, grpc.resource_quota=0x11dab1ed0, grpc.server_uri=dns:///datastore.googleapis.com:443}}: connect failed (UNKNOWN:connect: No route to host (65) {created_time:"2024-07-14T19:34:01.818588+01:00"}), backing off for 1000 ms
I0000 00:00:1720982043.308557  280621 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1720982043.313254  280621 check_gcp_environment_no_op.cc:29] ALTS: Platforms ot

Upserting datapoints MatchingEngineIndex index: projects/662868193247/locations/europe-west4/indexes/502362464684343296


INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Upserting datapoints MatchingEngineIndex index: projects/662868193247/locations/europe-west4/indexes/502362464684343296
I0000 00:00:1720982051.523019   31107 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1720982051.962361  270951 subchannel.cc:806] subchannel 0x11a665be0 {address=ipv6:%5B2a00:1450:4006:811::200a%5D:443, args={grpc.client_channel_factory=0x11dab2880, grpc.default_authority=europe-west4-aiplatform.googleapis.com:443, grpc.dns_enable_srv_queries=1, grpc.http2_scheme=https, grpc.internal.channel_credentials=0x11a6817b0, grpc.internal.client_channel_call_destination=0x1187cb598, grpc.internal.event_engine=0x108d93e90, grpc.internal.security_connector=0x11a697360, grpc.internal.subchannel_pool=0x108b8aaf0, grpc.max_receive_message_length=-1, grpc.max_send_message_length=-1, grpc.primary_user_agent=grpc-python/1.65.0, grpc.resource_quota=0x11dab1e

MatchingEngineIndex index Upserted datapoints. Resource name: projects/662868193247/locations/europe-west4/indexes/502362464684343296


I0000 00:00:1720982052.705164  270951 tcp_posix.cc:809] IOMGR endpoint shutdown
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex index Upserted datapoints. Resource name: projects/662868193247/locations/europe-west4/indexes/502362464684343296


['e8cc98c8-58fe-4076-9875-b9183b002e9f',
 '831a2c58-244a-4810-88ab-9e8110f83304',
 'f686949e-66bc-4a11-88b2-eb0ae5630fff',
 '835df3a9-021c-4f89-aad0-81d01416d530',
 '73d6be74-6346-415f-ad0f-c99754b7b53c']

In [35]:
from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import (
    Namespace,
    NumericNamespace,
)

AttributeError: 'VectorSearchVectorStoreDatastore' object has no attribute 'get_index'

In [36]:
# Try running a simple similarity search

# Below code should return 5 results
vector_store.similarity_search("shirt", k=5)

[Document(metadata={'season': ['summer', 'spring'], 'color': 'white', 'price': 34.99}, page_content='A lightweight linen button-down shirt in a crisp white. Perfect for keeping cool with breathable fabric and a relaxed fit.'),
 Document(metadata={'season': ['fall', 'winter', 'summer', 'spring'], 'color': 'blue', 'price': 19.99}, page_content='A classic crewneck t-shirt in a soft, heathered blue. Made from comfortable cotton jersey, this t-shirt is a wardrobe essential that works for every season.'),
 Document(metadata={'season': ['spring', 'summer'], 'color': 'white', 'price': 45.0}, page_content='A flowing midi-skirt in a delicate floral print. Lightweight and airy, this skirt adds a touch of feminine style to warmer days.'),
 Document(metadata={'season': ['fall', 'winter', 'spring'], 'color': 'blue', 'price': 65.0}, page_content='A versatile pair of dark-wash denim jeans.Made from durable cotton with a classic straight-leg cut, these jeans transition easily from casual days to dressi

I0000 00:00:1720798793.749594  871447 tcp_posix.cc:809] IOMGR endpoint shutdown


In [38]:
# Try running a similarity search with combination of text and numeric filter
filters = [Namespace(name="season", allow_tokens=["spring"])]
numeric_filters = [NumericNamespace(name="price", value_float=40.0, op="LESS")]

# Below code should return 2 results now
vector_store.similarity_search(
    "shirt", k=5, filter=filters, numeric_filter=numeric_filters
)

[Document(metadata={'season': ['summer', 'spring'], 'color': 'white', 'price': 34.99}, page_content='A lightweight linen button-down shirt in a crisp white. Perfect for keeping cool with breathable fabric and a relaxed fit.'),
 Document(metadata={'season': ['fall', 'winter', 'summer', 'spring'], 'color': 'blue', 'price': 19.99}, page_content='A classic crewneck t-shirt in a soft, heathered blue. Made from comfortable cotton jersey, this t-shirt is a wardrobe essential that works for every season.')]

I0000 00:00:1720799093.941880  871603 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1720799093.946460  871603 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1720799464.309203  871447 tcp_posix.cc:809] IOMGR endpoint shutdown


In [39]:
# Initialize the vectore_store as retriever
retriever = vector_store.as_retriever()

In [40]:
# perform simple similarity search on retriever
retriever.invoke("What are my options in breathable fabric?")

[Document(metadata={'season': ['summer', 'spring'], 'color': 'white', 'price': 34.99}, page_content='A lightweight linen button-down shirt in a crisp white. Perfect for keeping cool with breathable fabric and a relaxed fit.'),
 Document(metadata={'season': ['fall', 'winter'], 'color': 'green', 'price': 89.99}, page_content='A soft, chunky knit sweater in a vibrant forest green. The oversized fit and cozy wool blend make this ideal for staying warm when the temperature drops.'),
 Document(metadata={'season': ['fall', 'winter', 'summer', 'spring'], 'color': 'blue', 'price': 19.99}, page_content='A classic crewneck t-shirt in a soft, heathered blue. Made from comfortable cotton jersey, this t-shirt is a wardrobe essential that works for every season.'),
 Document(metadata={'season': ['spring', 'summer'], 'color': 'white', 'price': 45.0}, page_content='A flowing midi-skirt in a delicate floral print. Lightweight and airy, this skirt adds a touch of feminine style to warmer days.')]

In [41]:
from langchain_google_vertexai import VertexAI

llm = VertexAI(model_name="gemini-pro")

In [42]:
from langchain.chains import RetrievalQA

filters = [Namespace(name="season", allow_tokens=["spring"])]
numeric_filters = [NumericNamespace(name="price", value_float=40.0, op="LESS")]

retriever.search_kwargs = {"k": 2, "filter": filters, "numeric_filter": numeric_filters}

retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

question = "What are my options in breathable fabric?"
response = retrieval_qa({"query": question})
print(f"{response['result']}")
print("REFERENCES")
print(f"{response['source_documents']}")

/Users/abderrazzak/Desktop/rag_gcp/raggcp/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
I0000 00:00:1720799680.839160  707106 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


You have two options in breathable fabric: the lightweight linen button-down shirt and the classic crewneck t-shirt. 
 
 * The linen button-down shirt is made from crisp white linen, which is known for being a breathable and lightweight fabric. 
 * The classic crewneck t-shirt is made from soft, heathered blue cotton jersey, which is also a breathable and comfortable fabric. 
 
 Both of these shirts would be a good choice for staying cool in warm weather. The linen shirt is a bit more dressy than the t-shirt, so it might be a better choice if you are going to be somewhere where you need to look a little more put together. The t-shirt is more casual, so it would be a good choice for everyday wear.
REFERENCES
[Document(metadata={'season': ['summer', 'spring'], 'color': 'white', 'price': 34.99}, page_content='A lightweight linen button-down shirt in a crisp white. Perfect for keeping cool with breathable fabric and a relaxed fit.'), Document(metadata={'season': ['fall', 'winter', 'summer'

I0000 00:00:1720799924.665824  871447 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1720799924.666682  871447 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1720799924.667270  871447 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1720800294.946795  871447 tcp_posix.cc:809] IOMGR endpoint shutdown


In [43]:
PARENT = f"projects/{PROJECT_ID}/locations/{REGION}"

In [48]:
from google.cloud import aiplatform_v1 as aipv1
from google.api_core.client_options import ClientOptions

ENDPOINT = f"{REGION}-aiplatform.googleapis.com"

In [49]:
index_client = aipv1.IndexServiceClient(
            client_options=ClientOptions(api_endpoint=ENDPOINT)
        )

I0000 00:00:1720800590.808494  707106 work_stealing_thread_pool.cc:321] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1720800591.284697  707106 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [50]:
page_result = index_client.list_indexes(
            request=aipv1.ListIndexesRequest(parent=PARENT)
        )

In [54]:
indexes = [
            response.name
            for response in page_result
            if response.display_name == DISPLAY_NAME
        ]


In [55]:
indexes

['projects/662868193247/locations/us-central1/indexes/1270309764034723840']

I0000 00:00:1720800840.276845  903962 tcp_posix.cc:809] IOMGR endpoint shutdown


## OOP

In [26]:

from google.cloud import aiplatform
from langchain_google_vertexai import (
   VectorSearchVectorStoreDatastore
)

import numpy as np
from typing import Optional
from rag.logger import logger


from google.cloud import aiplatform_v1 as aipv1
from google.api_core.client_options import ClientOptions


class VertexAIVectorStore:
    def __init__(self, project_id: str,
                 region: str,
                 index_name: str,
                 index_endpoint_name: Optional[str] = None,
                 dimensions: int = 768 
                 ):
        """
        Initializes a new instance of the Raggcp class.

        Args:
            project_id (str): The ID of the Google Cloud project.
            region (str): The region where the index and index endpoint will be created.
            index_name (str): The name of the index.
            index_endpoint_name (Optional[str], optional): The name of the index endpoint. If not provided, it will be set to "{index_name}_endpoint". Defaults to None.
            dimensions (int, optional): The number of dimensions for the index. Defaults to 768.
        """
        
        self.project_id = project_id
        self.region = region
        self.dimensions = dimensions
        self.index_name = index_name
        self.index_endpoint_name = index_endpoint_name or f"{self.index_name}_endpoint"
        self.PARENT = f"projects/{self.project_id}/locations/{self.region}"

        ENDPOINT = f"{self.region}-aiplatform.googleapis.com"

        # set index client
        self.index_client = aipv1.IndexServiceClient(
            client_options=ClientOptions(api_endpoint=ENDPOINT)
        )
        # set index endpoint client
        self.index_endpoint_client = aipv1.IndexEndpointServiceClient(
            client_options=ClientOptions(api_endpoint=ENDPOINT)
        )

    def get_index(self):
        """
        Retrieves the index with the specified name.

        Returns:
            The index with the specified name, or None if it doesn't exist.
            :rtype: google.cloud.aiplatform_v1.types.index.Index or None
        """
        # Check if index exists
        page_result = self.index_client.list_indexes(
            request=aipv1.ListIndexesRequest(parent=self.PARENT)
        )
        indexes = [
            response.name
            for response in page_result
            if response.display_name == self.index_name
        ]

        if len(indexes) == 0:
            return None

        index_id = indexes[0]
        return self.index_client.get_index(request=aipv1.GetIndexRequest(name=index_id))
    
    def get_index_endpoint(self):
        """
        Retrieves the index endpoint with the specified name.

        Returns:
            The index endpoint object if found, None otherwise.
            :rtype: google.cloud.aiplatform_v1.types.index.IndexEndpoint or None
        """
        
        # Check if index endpoint exists
        page_result = self.index_endpoint_client.list_index_endpoints(
            request=aipv1.ListIndexEndpointsRequest(parent=self.PARENT)
        )
        index_endpoints = [
            response.name
            for response in page_result
            if response.display_name == self.index_endpoint_name
        ]

        if len(index_endpoints) == 0:
            return None

        index_endpoint_id = index_endpoints[0]
        return self.index_endpoint_client.get_index_endpoint(
            request=aipv1.GetIndexEndpointRequest(name=index_endpoint_id)
        )

    def delete_index(self):
        """
        Deletes the matching engine index.
        """

        # Check if index exists
        index = self.get_index()

        # create index if does not exists
        if index:
            # Delete index
            index_id = index.name
            logger.info(f"Deleting Index {self.index_name} with id {index_id}")
            self.index_client.delete_index(name=index_id)
        else:
            raise Exception("Index {index_name} does not exists.")

    def delete_index_endpoint(self):
        """
        Deletes the matching engine index endpoint.
        """

        # Check if index endpoint exists
        index_endpoint = self.get_index_endpoint()

        # Create Index Endpoint if does not exists
        if index_endpoint:
            logger.info(
                f"Index endpoint {self.index_endpoint_name}  exists with resource "
                + f"name as {index_endpoint.name} and endpoint domain name as "
                + f"{index_endpoint.public_endpoint_domain_name}"
            )

            index_endpoint_id = index_endpoint.name
            index_endpoint = self.index_endpoint_client.get_index_endpoint(
                name=index_endpoint_id
            )
            # Undeploy existing indexes
            for d_index in index_endpoint.deployed_indexes:
                logger.info(
                    f"Undeploying index with id {d_index.id} from Index endpoint {self.index_endpoint_name}"
                )
                request = aipv1.UndeployIndexRequest(
                    index_endpoint=index_endpoint_id, deployed_index_id=d_index.id
                )
                r = self.index_endpoint_client.undeploy_index(request=request)
                response = r.result()
                logger.info(response)

            # Delete index endpoint
            logger.info(
                f"Deleting Index endpoint {self.index_endpoint_name} with id {index_endpoint_id}"
            )
            self.index_endpoint_client.delete_index_endpoint(name=index_endpoint_id)
        else:
            raise Exception(
                f"Index endpoint {self.index_endpoint_name} does not exists."
            )
        
    def delete_all(self):
        """
        Deletes the matching engine index and endpoint.
        """
        self.delete_index_endpoint()
        self.delete_index()

    def create_index(self):
        """
        Creates a matching engine index with the specified parameters.

        Returns:
            The created matching engine index.
            :rtype: google.cloud.aiplatform.matching_engine.matching_engine_index.MatchingEngineIndex

        """

        # Get index
        index = self.get_index()
        # Create index if does not exists
        if index:
            logger.info(f"Index {self.index_name} already exists with id {index.name}")
            return aiplatform.MatchingEngineIndex(index_name=index.name)
        
        logger.info(f"Index {self.index_name} does not exists. Creating index ...")
        index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
            display_name=self.index_name,
            location=self.region,
            dimensions=self.dimensions,
            approximate_neighbors_count=150,
            distance_measure_type="DOT_PRODUCT_DISTANCE",
            index_update_method="STREAM_UPDATE",  # allowed values BATCH_UPDATE , STREAM_UPDATE
        )

        logger.info(f"Index created: {index.display_name}")
        return index

    def create_endpoint(self):
        """
        Creates an endpoint for the matching engine index.

        Returns:
            The created index endpoint.
            :rtype: google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint.MatchingEngineIndexEndpoint
        """

        # Get index endpoint if exists
        index_endpoint = self.get_index_endpoint()
        # Create Index Endpoint if does not exists
        if index_endpoint:
            logger.info(
                f"Index endpoint {self.index_endpoint_name} already exists with resource "
                + f"name as {index_endpoint.name} and endpoint domain name as "
                + f"{index_endpoint.public_endpoint_domain_name}"
            )
            return aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name=index_endpoint.name)
        
        logger.info(
            f"Index endpoint {self.index_endpoint_name} does not exists. Creating index endpoint..."
        )
        # Create an endpoint
        index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
            display_name=self.index_endpoint_name, 
            location=self.region,
            public_endpoint_enabled=True
        )

        logger.info(f"Endpoint created: {index_endpoint.display_name}")
        return index_endpoint

    def deploy(self):
        # NOTE : This operation can take upto 20 minutes
        """
        Deploys the index to the endpoint.

        Returns:
            None
        """
        # Deploy index to endpoint
        index = self.create_index()
        index_endpoint = self.create_endpoint()
        deployed_index_id = index_endpoint.display_name

        index_endpoint = index_endpoint.deploy_index(
            index=index, 
            deployed_index_id=deployed_index_id
        )
        logger.info(f"Index deployed to endpoint: {index_endpoint.display_name}")
        logger.info(f"Deployed indexes: {index_endpoint.deployed_indexes}")

    def get_vector_store(self, embedding_model, datastore_client_kwargs):
        # https://python.langchain.com/v0.2/docs/integrations/retrievers/google_vertex_ai_search/

        # add filters
        # https://github.com/langchain-ai/langchain/issues/5073

        """
            Parameters:
            - embedding_model: The embedding model used for vectorization.
            - datastore_client_kwargs: The datastore client kwargs.

            Returns:
            - vector_store: The vector store object for the matching engine index.
        """

        vector_store = VectorSearchVectorStoreDatastore.from_components(
            project_id=self.project_id,
            region=self.region,
            index_id=self.get_index().name,
            endpoint_id=self.get_index_endpoint().name,
            embedding=embedding_model,
            datastore_client_kwargs=datastore_client_kwargs,
            stream_update=True,
        )
        return vector_store

    def upsert(self, texts, metadatas, embedding_model, datastore_client_kwargs):
        # create datastore first 
        """
        Upserts the given texts and metadatas into the specified index.

        Parameters:
        - texts (list): A list of texts to be upserted.
        - metadatas (list): A list of metadata associated with each text.
        - index_endpoint (IndexEndpoint): The index endpoint to upsert the data into.
        - index (Index): The index to upsert the data into.
        - embedding_model: The embedding model used for vectorization.
        - datastore_client_kwargs: The datastore client kwargs.

        Returns:
        None
        """
        
        vector_store = self.get_vector_store(embedding_model, datastore_client_kwargs)
        vector_store.add_texts(texts=texts, metadatas=metadatas, is_complete_overwrite=True)
        logger.info("Upserted data to the index.")

    def retrieve(self, embedding_model, datastore_client_kwargs):
        """
        Retrieves the vector store using the specified embedding model and datastore client arguments.

        Parameters:
        - embedding_model: The embedding model to use for retrieval.
        - datastore_client_kwargs: The keyword arguments to pass to the datastore client.

        Returns:
        - retriever: The retriever object for the vector store.
        """
        vector_store = self.get_vector_store(embedding_model, datastore_client_kwargs)
        return vector_store.as_retriever()


In [19]:
# Input text with metadata
record_data = [
    {
        "description": "A versatile pair of dark-wash denim jeans."
        "Made from durable cotton with a classic straight-leg cut, these jeans"
        " transition easily from casual days to dressier occasions.",
        "price": 65.00,
        "color": "blue",
        "season": ["fall", "winter", "spring"],
    },
    {
        "description": "A lightweight linen button-down shirt in a crisp white."
        " Perfect for keeping cool with breathable fabric and a relaxed fit.",
        "price": 34.99,
        "color": "white",
        "season": ["summer", "spring"],
    },
    {
        "description": "A soft, chunky knit sweater in a vibrant forest green. "
        "The oversized fit and cozy wool blend make this ideal for staying warm "
        "when the temperature drops.",
        "price": 89.99,
        "color": "green",
        "season": ["fall", "winter"],
    },
    {
        "description": "A classic crewneck t-shirt in a soft, heathered blue. "
        "Made from comfortable cotton jersey, this t-shirt is a wardrobe essential "
        "that works for every season.",
        "price": 19.99,
        "color": "blue",
        "season": ["fall", "winter", "summer", "spring"],
    },
    {
        "description": "A flowing midi-skirt in a delicate floral print. "
        "Lightweight and airy, this skirt adds a touch of feminine style "
        "to warmer days.",
        "price": 45.00,
        "color": "white",
        "season": ["spring", "summer"],
    },
]



# Parse and prepare input data

texts = []
metadatas = []
for record in record_data:
    record = record.copy()
    page_content = record.pop("description")
    texts.append(page_content)
    if isinstance(page_content, str):
        metadata = {**record}
        metadatas.append(metadata)

datastore_client_kwargs = {
    'database': 'orangeds',
    # add other necessary parameters here
}

In [20]:
import os

os.environ['PATH'] += ':/Users/abderrazzak/google-cloud-sdk/bin'

os.environ['REGION'] = "europe-west4"
os.environ['PROJECT_ID'] = "orangebot-7551"
os.environ['BUCKET_URI'] = 'gs://orange_maroc'

REGION = os.environ['REGION']
PROJECT_ID = os.environ['PROJECT_ID']
BUCKET_URI = os.environ['BUCKET_URI']

# The number of dimensions for the textembedding-gecko@003 is 768
# If other embedder is used, the dimensions would probably need to change.
DIMENSIONS = 768

# Index Constants
DISPLAY_NAME = "orange_maroc"
DEPLOYED_INDEX_ID = "orange_maroc_endpoint"

In [27]:
vector_store = VertexAIVectorStore(project_id=PROJECT_ID, region=REGION, index_name=DISPLAY_NAME, index_endpoint_name=DEPLOYED_INDEX_ID, dimensions=DIMENSIONS)

I0000 00:00:1721034604.226915    8819 work_stealing_thread_pool.cc:321] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1721034604.725539    8819 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1721034604.727733    8819 work_stealing_thread_pool.cc:321] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1721034604.936737    8819 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [15]:
#vector_store.deploy()

In [29]:
vector_store.upsert(texts, metadatas, embedding_model, datastore_client_kwargs)

I0000 00:00:1721039595.062967    8819 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1721039595.618394   49582 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1721039595.621409    8819 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1721039596.046025   49584 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1721039596.050146    8819 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1721039596.054919    8819 work_stealing_thread_pool.cc:321] WorkStealingThreadPoolImpl::PrepareFork


Upserting datapoints MatchingEngineIndex index: projects/662868193247/locations/europe-west4/indexes/3773101694062166016


I0000 00:00:1721039598.307658    8819 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


MatchingEngineIndex index Upserted datapoints. Resource name: projects/662868193247/locations/europe-west4/indexes/3773101694062166016


I0000 00:00:1721039598.866818  129852 tcp_posix.cc:809] IOMGR endpoint shutdown
INFO:logger:Upserted data to the index.
I0000 00:00:1721039598.869383  129853 tcp_posix.cc:809] IOMGR endpoint shutdown


In [23]:
retriever = vector_store.retrieve(embedding_model, datastore_client_kwargs)

I0000 00:00:1721034032.124761    8819 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1721034032.510671   40547 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1721034032.511942    8819 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1721034032.877357   40542 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1721034032.877904    8819 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1721034032.880138    8819 work_stealing_thread_pool.cc:321] WorkStealingThreadPoolImpl::PrepareFork


In [30]:
retriever.invoke("What are my options in breathable fabric?")

ServiceUnavailable: 503 DNS resolution failed for 1823754328.europe-west4-662868193247.vdb.vertexai.goog:443: C-ares status is not ARES_SUCCESS qtype=SRV name=_grpclb._tcp.1823754328.europe-west4-662868193247.vdb.vertexai.goog: Domain name not found

I0000 00:00:1721051223.098926  134295 tcp_posix.cc:809] IOMGR endpoint shutdown


In [28]:
vector_store.delete_all()

INFO:logger:Index endpoint orange_maroc_endpoint  exists with resource name as projects/662868193247/locations/europe-west4/indexEndpoints/6559915069730193408 and endpoint domain name as 1823754328.europe-west4-662868193247.vdb.vertexai.goog
INFO:logger:Undeploying index with id orange_maroc_endpoint from Index endpoint orange_maroc_endpoint
INFO:logger:
INFO:logger:Deleting Index endpoint orange_maroc_endpoint with id projects/662868193247/locations/europe-west4/indexEndpoints/6559915069730193408
INFO:logger:Deleting Index orange_maroc with id projects/662868193247/locations/europe-west4/indexes/502362464684343296


I0000 00:00:1721034653.218987   49675 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1721034793.325641   49675 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1721034863.384886   49576 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1721034863.385607   49576 tcp_posix.cc:809] IOMGR endpoint shutdown
